<a href="https://colab.research.google.com/github/AnanthSrinivasan/TickerFetcherDemo/blob/main/Ticker_Fetcher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installing the libraries with the specified version.
!pip install pandas==2.2.2 numpy==1.25.2 requests beautifulsoup4 gradio -q --user

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 83.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.1 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-s

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
# import tempfile

def fetch_data_from_url_with_pagination(base_url, max_pages=5):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    session = requests.Session()
    session.headers.update(headers)

    combined_data = []

    for page in range(1, max_pages + 1):
        # Add pagination only if page > 1 (to handle single-page URLs)
        if page == 1:
            url = base_url
        else:
            url = f"{base_url}&r={(page - 1) * 20 + 1}"

        response = session.get(url)

        if response.status_code != 200:
            print(f"Failed to fetch page: {page} (status: {response.status_code})")
            break

        soup = BeautifulSoup(response.text, 'html.parser')
        rows = soup.select('tr[valign="top"]')

        # Stop if no rows are found
        if not rows:
            print(f"No data found on page {page}. Stopping.")
            break

        # Process rows
        for row in rows:
            cells = row.find_all('td')
            if len(cells) == 11:  # Ensure valid row with 11 columns
                combined_data.append([cell.text.strip() for cell in cells])

        # If it’s a single-page URL, stop after the first page
        if page == 1 and len(rows) < 20:  # Assuming <20 rows means no pagination
            break

    # Define columns
    columns = [
        'No.', 'Ticker', 'Company', 'Sector', 'Industry',
        'Country', 'Market Cap', 'P/E', 'Price', 'Change', 'Volume'
    ]

    # Create DataFrame
    df = pd.DataFrame(combined_data, columns=columns)

    # Remove duplicate tickers
    df = df.drop_duplicates(subset=['Ticker'], keep='first')

    tickers = df['Ticker'].tolist()


    # Convert the tickers list to a string with each ticker on a new line
    tickers_str = "\n".join(tickers)
    return tickers_str

    # # Create a temporary file
    # with tempfile.NamedTemporaryFile(mode="w+", delete=False, suffix=".txt") as tmp_file:
    #     tmp_file.write(tickers_str)
    #     file_path = tmp_file.name  # Get the path of the temporary file

    # return file_path  # Return the file path


In [ ]:
import gradio as gr

# creating the deployment input interface
model_inputs = [
    gr.Textbox(label="Url")
]

# creating the deployment output interface
model_outputs = [
    gr.Textbox(label="Tickers...")
]

# # creating the deployment output interface
# model_outputs = [
#     gr.File(label="Download Tickers") # Use gr.File instead of gr.Textbox
# ]

# defining the structure of the deployment interface and how the components will interact
demo = gr.Interface(
    fn = fetch_data_from_url_with_pagination,
    inputs = model_inputs,
    outputs = model_outputs,
    allow_flagging='never',
    title = "Ticker Fetcher",
    description = "This interface will fetch the tickers for the given Finviz url.",
)

/root/.local/lib/python3.10/site-packages/gradio/interface.py:399: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


In [ ]:
# deploying the model
demo.launch(inline=False, share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://dc720b0b9c4d482151.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://dc720b0b9c4d482151.gradio.live


In [ ]:
demo.close()

Closing server running on port: 7860
